In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import pyodbc
import warnings
from matplotlib.lines import Line2D
import scipy.stats
import matplotlib.ticker as mtick
warnings.filterwarnings('ignore')

from IPython.display import HTML
config = dict(server='ABR-RIPLEYA-D1',
    port=      1433,
    database=  'boydcartridgePQ2022',
    )

conn_str = ('SERVER={server};' +
            'Database={database};' +
            'TRUSTED_CONNECTION=yes')

conn = pyodbc.connect(
    r'DRIVER={SQL Server};' +
    conn_str.format(**config)
    )

In [2]:
colorDict  = {1:'#FF0000',#Red 1
              2:'#00B050',#Green 2
              3:'#0070C0',#`Blue 3
              4:'#7030A0',#Purple 4
              5:'#808080',#Light Grey 5
              6:'#FF6600',#Orange 6
              7:'#FFCC00',#Yellow 7
              8:'#9999FF',#Light Purple 8
              9:'#333333',#Black 9
              10:'#808000',#Goldish 10
              11:'#FF99CC',#Hot Pink 11
              12:'#003300',#Dark Green 12
            }

##Function to Get Channel Data (i.e. Green Raw, Red Normalized etc.)
def getSQLData_Channel(color, dtype, assay, start_time_filter, end_date_filter,system,assay_version,cartridge_lot_filter, cartridge_lot_filter_b, cartridge_lot_filter_c, cartridge_lot_filter_d):
    query = ("Select * from [dbo].[NeuMoDx_"+color+"_"+dtype+"_Master]"+
             "where ([Result Code] like '%"+assay+"%') and [Start Time]>'"+start_time_filter+"'"+
             " and [Start Time] < '"+end_date_filter+"' and [N500 Serial Number] like '%"+system+"%'"+
             " and [Assay Version] like '%"+assay_version+
             "%' and ([Cartridge Barcode] like '%"+cartridge_lot_filter+
             "%' OR [Cartridge Barcode] like '%"+cartridge_lot_filter_b+
             "%' OR [Cartridge Barcode] like '%"+cartridge_lot_filter_c+
             "%' OR [Cartridge Barcode] like '%"+cartridge_lot_filter_d+"%')")
    print("Query:", query)
    df = pd.read_sql(query, conn)
    return df


##Function to Get Channel Data (i.e. Green Raw, Red Normalized etc.)             
def getSQLData_COC(assay, start_time_filter, end_date_filter, system, assay_version, cartridge_lot_filter, cartridge_lot_filter_b, cartridge_lot_filter_c, cartridge_lot_filter_d):
    query = ("Select * from [dbo].[NeuMoDx_System_Master_Table]"+
             " where ([Result Code] like '%"+assay+"%') and [Start Date Time]>'"+start_time_filter+"'"+
             " and [Start Date Time] < '"+end_date_filter+"' and [N500 Serial Number] like '%"+system+"%'"+
             " and [Assay Version] like '%"+assay_version+"%' and ([Pcr Cartridge Barcode] like '%"+cartridge_lot_filter+
             "%' OR [Pcr Cartridge Barcode] like '%"+cartridge_lot_filter_b+
             "%' OR [Pcr Cartridge Barcode] like '%"+cartridge_lot_filter_c+
             "%' OR [Pcr Cartridge Barcode] like '%"+cartridge_lot_filter_d+"%')")
    print("Query:", query)
    df = pd.read_sql(query, conn)
    
    return df.set_index(['PCR Module Serial', 'Pcr Cartridge Lane'])


NormDataDict_PCR = {}
RawDataDict_PCR = {}
SecondDataDict_PCR = {}


##getSQLData_COC args (Result Code, Start Date, End Date, N500 Serial Number, cartridge lot 1, cartridge lot 2, cartridge lot 3, cartridge lot 4)
COC_PCR = getSQLData_COC('COV1', '2022-04-01', '2022-12-01', '', '', '', '', '', '')


##getSQLData_Channel args (Result Code, Channel, Start Date, End Date, N500 Serial Number, Assay Version, cartridge lot 1, cartridge lot 2, cartridge lot 3, cartridge lot 4)
for channel in ['Green','Yellow', 'Orange', 'Far_Red', 'Red']:
    
    NormDataDict_PCR[channel] = getSQLData_Channel(channel,'Normalized','COV1', '2022-04-01', '2022-12-01', '', '', '', '11681Y', '11681Y', '11681Y')
    RawDataDict_PCR[channel] = getSQLData_Channel(channel,'Raw', 'COV1', '2022-04-01', '2022-12-01', '', '', '', '11681Y', '11681Y', '11681Y')
    SecondDataDict_PCR[channel]= getSQLData_Channel(channel,'2nd','COV1', '2022-04-01', '2022-12-01', '', '', '', '11681Y', '11681Y', '11681Y')
    NormDataDict_PCR[channel].set_index(['XPCR Module Serial','Cartridge Lane',  'Test Guid'],inplace=True)
    RawDataDict_PCR[channel].set_index(['XPCR Module Serial','Cartridge Lane', 'Test Guid'],inplace=True)
    SecondDataDict_PCR[channel].set_index(['XPCR Module Serial','Cartridge Lane', 'Test Guid'],inplace=True)
    NormDataDict_PCR[channel].sort_index(inplace=True)

Query: Select * from [dbo].[NeuMoDx_System_Master_Table] where ([Result Code] like '%COV1%') and [Start Date Time]>'2022-04-01' and [Start Date Time] < '2022-12-01' and [N500 Serial Number] like '%%' and [Assay Version] like '%%' and ([Pcr Cartridge Barcode] like '%%' OR [Pcr Cartridge Barcode] like '%%' OR [Pcr Cartridge Barcode] like '%%' OR [Pcr Cartridge Barcode] like '%%')
Query: Select * from [dbo].[NeuMoDx_System_Master_Table] where ([Result Code] like '%COV1%') and [Start Date Time]>'2022-04-01' and [Start Date Time] < '2022-12-01' and [N500 Serial Number] like '%%' and [Assay Version] like '%%' and ([Pcr Cartridge Barcode] like '%%' OR [Pcr Cartridge Barcode] like '%%' OR [Pcr Cartridge Barcode] like '%%' OR [Pcr Cartridge Barcode] like '%%')
Query: Select * from [dbo].[NeuMoDx_Green_Normalized_Master]where ([Result Code] like '%COV1%') and [Start Time]>'2022-04-01' and [Start Time] < '2022-12-01' and [N500 Serial Number] like '%%' and [Assay Version] like '%%' and ([Cartridge

In [4]:
##Returns Consumable Information for each module included in Dataset Takes Chain of Custody Data as input
def generateLotTable(data):
    LotDataBase = data.sort_values('Start Date Time')

    #stats = ['Ct', 'End Point Fluorescence', 'Max Peak Height', 'EPR']
    #results_Stats = ['Amp','NotAmp','NoResult','Sample Size']
    #LotDataBase['Run Number'] = np.nan
    for mod in LotDataBase.index.unique(0):
        LotDataBase.loc[mod, 'Time Difference'] = LotDataBase.loc[mod, 'Start Date Time'].diff().values / np.timedelta64(1, 's')
        LotDataBase.loc[mod, 'Time Difference'] = np.where(LotDataBase.loc[mod, 'Time Difference'].isnull(),50000,LotDataBase.loc[mod, 'Time Difference'])
        LotDataBase.loc[mod, 'New Run?'] = np.where(LotDataBase.loc[mod, 'Time Difference']>3600,True,False) 
        LotDataBase.loc[(mod, (LotDataBase['New Run?']==1)), 'Run Number'] = np.arange(1, len(LotDataBase.loc[mod, LotDataBase['New Run?']==1,:])+1)
        LotDataBase.loc[mod,['Run Number','Start Date Time','Time Difference','New Run?']] = LotDataBase.loc[mod,['Run Number','Start Date Time','Time Difference','New Run?']].fillna(method='ffill').values

    LotDataBase.reset_index(inplace=True)
    LotDataBase['Date'] = LotDataBase['Start Date Time'].dt.date
    LotDataBase.set_index(['PCR Module Serial', 'Run Number', 'Date'],inplace=True)
    LotDataBase['Cartridge Lot'] = LotDataBase['Pcr Cartridge Barcode'].str[18:24]
    LotDataBase['Cartridge Serial'] = LotDataBase['Pcr Cartridge Barcode'].str[27:32]
    LotDataBase['Test Strip Lot'] = LotDataBase['Test Strip NeuMoDx Barcode'].str[18:24]
    LotDataBase['LDT Master Mix Lot'] = LotDataBase['Test Strip LDT Master Mix Barcode'].str[18:24]
    LotDataBase['Extraction Plate Lot'] = LotDataBase['Capture Plate Barcode'].str[18:24]
    LotDataBase['Buffer Lot'] = LotDataBase['Buffer Barcode'].str[18:24]                                                 
    LotDataBase['Release Lot'] = LotDataBase['Release Reagent Barcode'].str[18:24]
    LotDataBase['Wash Lot'] = LotDataBase['Wash Reagent Barcode'].str[18:24]
    


    Lots = ['Cartridge Lot', 'Cartridge Serial', 'Test Strip Lot', 'Extraction Plate Lot', 'Buffer Lot', 'Release Lot',  'Wash Lot', 'Result Code']
    #display(LotDataBase['LotDataBase Strip Lot'].groupby(level=[0,1]).agg('unique'))
    LotDataBase_aggs = {}
    for lot_type in Lots:
        LotDataBase_by_run_agg = LotDataBase[lot_type].groupby(level=[0,1,2]).agg('unique').to_frame()
        LotDataBase_by_mod_agg = LotDataBase[lot_type].groupby(level=[0]).agg('unique').to_frame()
        LotDataBase_by_date_agg = LotDataBase[lot_type].groupby(level=[2]).agg('unique').to_frame()
        #LotDataBase_by_date_agg = LotDataBase[lot_type].groupby(level=[2]).agg('unique').to_frame()    

        LotDataBase_by_mod_agg['Run Number'] = 'Combined'
        LotDataBase_by_mod_agg['Date'] = 'Combined'
        
        LotDataBase_by_date_agg['Run Number'] = 'Combined'
        LotDataBase_by_date_agg['PCR Module Serial'] = 'Combined'
        
        LotDataBase_by_mod_agg.set_index(['Run Number','Date'],append=True,inplace=True)
        LotDataBase_by_date_agg.set_index(['Run Number', 'PCR Module Serial'],append=True,inplace=True)
        LotDataBase_by_date_agg = LotDataBase_by_date_agg.reorder_levels(['PCR Module Serial', 'Run Number', 'Date'],axis=0)
        LotDataBase_agg = pd.concat([LotDataBase_by_run_agg,LotDataBase_by_mod_agg,LotDataBase_by_date_agg],axis=0).sort_index()
        #LotDataBase_agg.sort_index(axis=1,inplace=True)
        #LotDataBase_agg.set_index('Ct Calls',append=True,inplace=True)
        LotDataBase_aggs[lot_type] = LotDataBase_agg
    consumables = pd.concat([LotDataBase_aggs[x] for x in LotDataBase_aggs],axis=1)
    consumables.index.names = ['XPCR Module Serial', 'Run Number','Date']
    consumables.columns = pd.MultiIndex.from_tuples([(lot, '', '') for lot in consumables.columns])
    return consumables, LotDataBase[Lots+['Test Guid', 'LHPA ADP Position', 'LHPB ADP Position', 'LHPC ADP Position', 'Pcr Cartridge Lane']]

##Returns By Run Summary Information & Run Labeled Line Data for each module included in Dataset Takes Channel Data as input
def generateSummaryTable(data, channel):
    SummaryDataBase = data.sort_values('Start Time')

    stats = ['Ct', 'End Point Fluorescence', 'Max Peak Height', 'EPR']
    results_Stats = ['Amp','NotAmp','NoResult','IND', 'UNR', 'Sample Size']
    #SummaryDataBase['Run Number'] = np.nan
    for mod in SummaryDataBase.index.unique(0):
        SummaryDataBase.loc[mod, 'Time Difference'] = SummaryDataBase.loc[mod, 'Start Time'].diff().values / np.timedelta64(1, 's')
        SummaryDataBase.loc[mod, 'Time Difference'] = np.where(SummaryDataBase.loc[mod, 'Time Difference'].isnull(),50000,SummaryDataBase.loc[mod, 'Time Difference'])
        SummaryDataBase.loc[mod, 'New Run?'] = np.where(SummaryDataBase.loc[mod, 'Time Difference']>3600,True,False) 
        SummaryDataBase.loc[(mod, (SummaryDataBase['New Run?']==1)), 'Run Number'] = np.arange(1, len(SummaryDataBase.loc[mod, SummaryDataBase['New Run?']==1,:])+1)
        SummaryDataBase.loc[mod,['Run Number','Start Time','Time Difference','New Run?']] = SummaryDataBase.loc[mod,['Run Number','Start Time','Time Difference','New Run?']].fillna(method='ffill').values

    SummaryDataBase['NotAmp'] = np.where(SummaryDataBase['Target Result']=='TargetNotAmplified',1,0)
    SummaryDataBase['Amp'] = np.where(SummaryDataBase['Target Result']=='TargetAmplified',1,0)
    SummaryDataBase['NoResult'] = np.where(SummaryDataBase['Target Result']=='NoResult',1,0)
    SummaryDataBase['IND'] = np.where(SummaryDataBase['Target Result']=='TargetIndeterminate',1,0)
    SummaryDataBase['UNR'] = np.where(SummaryDataBase['Target Result']=='TargetUnresolved',1,0)
    SummaryDataBase['Abort'] = np.where(SummaryDataBase['Target Result']=='TargetAborted',1,0)
    SummaryDataBase['Sample Size'] = SummaryDataBase[['NotAmp','Amp','NoResult','IND','UNR']].sum(axis=1)

    SummaryDataBase.loc[:,['NotAmp','Amp','IND','UNR','NoResult','Sample Size']]



    SummaryDataBase.reset_index(inplace=True)
    SummaryDataBase.set_index(['XPCR Module Serial','Run Number'],inplace=True)
    SummaryDataBase.loc[:,stats] =  SummaryDataBase.loc[:,stats].astype(float)

    SummaryDataBase.to_csv('SummaryDataBase.csv')

    agg_dict = {}
    agg_dict['Start Time'] = ['first','last']
    for stat in results_Stats:
        agg_dict[stat] = ['sum']
    for stat in stats:
        agg_dict[stat] = ['mean','std','count']
    agg_dict

    SummaryDataBase_by_run_agg = SummaryDataBase[stats+results_Stats+['Start Time']].groupby(level=[0,1]).agg(agg_dict)



    SummaryDataBase_by_mod_agg = SummaryDataBase[stats+results_Stats+['Start Time']].groupby(level=[0]).agg(agg_dict)


    SummaryDataBase_by_mod_agg['Run Number'] = 'Combined'
    SummaryDataBase_by_mod_agg.set_index('Run Number',append=True,inplace=True)

    SummaryDataBase_agg = pd.concat([SummaryDataBase_by_run_agg,SummaryDataBase_by_mod_agg],axis=0).sort_index()

    SummaryDataBase_agg[('Ct Calls','')] = SummaryDataBase_agg[('Ct','count')]
    for param in SummaryDataBase_agg.columns.unique(0):
        if param in stats:
            SummaryDataBase_agg[(param, '%CV')] = SummaryDataBase_agg[(param, 'std')] / SummaryDataBase_agg[(param, 'mean')]
            SummaryDataBase_agg.drop([(param,'count')],axis=1,inplace=True)
    SummaryDataBase_agg.sort_index(axis=1,inplace=True)
    SummaryDataBase_agg.set_index('Ct Calls',append=True,inplace=True)
    SummaryDataBase_agg.columns = pd.MultiIndex.from_tuples([(channel,x,y) for x,y in SummaryDataBase_agg.columns])
    SummaryDataBaseagg = SummaryDataBase_agg.loc[:,channel]
    SummaryDataBaseagg[('% Amp', '')] = SummaryDataBaseagg[('Amp', 'sum')] / (SummaryDataBaseagg[('Amp', 'sum')] + SummaryDataBaseagg[('NotAmp', 'sum')])
    SummaryDataBaseagg = SummaryDataBaseagg.loc[:,['Start Time', '% Amp', 'Amp','NotAmp','NoResult', 'IND', 'UNR', 'Sample Size', 'Ct', 'End Point Fluorescence', 'Max Peak Height', 'EPR']].droplevel(2,axis=0)
    SummaryDataBaseagg.columns = pd.MultiIndex.from_tuples([('Start Time', 'first sample'),
                                                 ('Start Time', 'last sample'),
                                                 ('% AMP', ''),
                                                 ('Amp', '') ,
                                                 ('NotAmp', '') ,
                                                 ('NoResult', ''),
                                                 ('UNR', '') ,
                                                 ('IND', '') ,
                                                 ('Sample Size', ''),
                                                 ('Ct', '%CV') ,
                                                 ('Ct', 'mean') ,
                                                 ('Ct', 'std') ,
                                                 ('End Point Fluorescence', '%CV') ,
                                                 ('End Point Fluorescence', 'mean') ,
                                                 ('End Point Fluorescence', 'std'),
                                                 ('Max Peak Height', '%CV') ,
                                                 ('Max Peak Height', 'mean') ,
                                                 ('Max Peak Height', 'std') ,
                                                 ('EPR', '%CV') ,
                                                 ('EPR', 'mean') ,
                                                 ('EPR', 'std')])
    return SummaryDataBaseagg, SummaryDataBase

In [5]:
mods = ['V949']
COC_PCR = COC_PCR.loc[([x for x in COC_PCR.index.unique(0) if pd.isnull(x)==False],
                       [x for x in COC_PCR.index.unique(1) if pd.isnull(x)==False]),:]

#DF = generateLotTable(COC_PCR).join(generateSummaryTable(RawDataDict_PCR['Yellow'],'Yellow'))
channelData = {}    

channelData['Green'] = generateSummaryTable(RawDataDict_PCR['Green'], 'Green')
channelData['Yellow'] =  generateSummaryTable(RawDataDict_PCR['Yellow'], 'Yellow')
channelData['Far_Red'] =  generateSummaryTable(RawDataDict_PCR['Far_Red'], 'Far_Red')

consumables = generateLotTable(COC_PCR)


In [6]:
##Step 1A Get Consumables used for Testing By Date.
display(HTML(consumables[0].loc[([x for x in consumables[0].index.unique(0) if x == 'Combined'],[y for y in consumables[0].index.unique(1) if y == 'Combined']),:].reset_index()\
     .style.hide_index()\
     .to_html()))



##Step 1B Get Consumables used for Testing By Module.
display(HTML(consumables[0].loc[([x for x in consumables[0].index.unique(0) if x != 'Combined'],[y for y in consumables[0].index.unique(1) if y == 'Combined']),:].reset_index()\
     .style.hide_index()\
     .to_html()))

##Step 1C Get Consumables used for Testing By Module Run.
display(HTML(consumables[0].loc[(slice(None),[x for x in consumables[0].index.unique(1) if x != 'Combined']),:].reset_index()\
     .style.hide_index()\
     .format(subset=['Run Number'], formatter='{:.0f}')\
     .to_html()))

XPCR Module Serial,Run Number,Date,Cartridge Lot,Cartridge Serial,Test Strip Lot,Extraction Plate Lot,Buffer Lot,Release Lot,Wash Lot,Result Code
,,,,,,,,,,
,,,,,,,,,,
Combined,Combined,2022-04-01,['118522'],['03387' '03388' '02893' '03396' '03389' '02894' '03393' '03392' '02896' '03391' '03390' '02895' '03386' '03384' '02890' '02889' '03376' '03394' '03184' '03185' '02892' '02891' '03186' '03187' '03151' '03152' '03188' '03189' '03190' '03191' '03192' '03193'],['114737'],['113600'],['116021'],['116045'],['116044'],['COV1']
Combined,Combined,2022-04-04,['118522'],['02671' '02670' '02283' '02238' '02672' '02231' '02230' '02673' '02235' '02236' '02237' '02232' '02888' '02857' '02250' '02251' '02957' '02234' '02233' '02956' '02332' '02333' '02334' '02340' '02677' '02676' '02336' '02337' '02339' '02338' '02669' '02678' '02208' '02171' '02202' '02335' '02654' '02653' '02675' '02674' '02900' '02899' '02898' '02897' '02903' '02904' '02905' '02906'],['114737' '' '114728'],['113600' '111478'],['116021'],['116045'],['116044'],['COV1']
Combined,Combined,2022-04-05,['118522'],['02866' '02867' '02901' '02902' '02648' '02649' '02650' '02651' '02644' '02645' '02646' '02647' '02617' '02616' '02642' '02643' '03395' '03373' '03374' '03375' '02435' '02434' '03234' '03237' '03238' '03240' '02436' '02437' '03084' '03083' '03236' '02441' '03235' '02440' '03239' '03241' '02433' '03242' '03243' '02442' '03379' '03372' '03403' '03380' '02367' '02366' '03385' '03397' '03378' '03377' '02439' '02438' '01734' '01733' '02016' '02015' '02031' '02030' '01732' '01731' '02002' '02003' '02004' '02005' '02037' '02036' '02000' '02001'],['114728' '114721'],['113600'],['116021'],['116045'],['116044'],['COV1']
Combined,Combined,2022-04-06,['118522'],['02021' '02022' '02023' '02024' '02017' '02018' '02019' '02020' '01996' '01740' '01739' '01997' '01998' '01999' '01804' '01735' '01736' '01907' '01805' '01903' '01906' '01902' '01737' '01905' '01904' '01738' '01901' '01900' '01880' '01879' '01681' '03253' '01909' '01908' '02420' '02416' '02206' '02710' '02415' '02417' '02422' '02421' '02067' '02935' '02414' '02413' '03284' '01963' '02300' '02483' '02419' '02418'],['114721' '114737'],['113600'],['116021'],['116045'],['116044'],['COV1']


XPCR Module Serial,Run Number,Date,Cartridge Lot,Cartridge Serial,Test Strip Lot,Extraction Plate Lot,Buffer Lot,Release Lot,Wash Lot,Result Code
,,,,,,,,,,
,,,,,,,,,,
Combined,Combined,2022-04-01,['118522'],['03387' '03388' '02893' '03396' '03389' '02894' '03393' '03392' '02896' '03391' '03390' '02895' '03386' '03384' '02890' '02889' '03376' '03394' '03184' '03185' '02892' '02891' '03186' '03187' '03151' '03152' '03188' '03189' '03190' '03191' '03192' '03193'],['114737'],['113600'],['116021'],['116045'],['116044'],['COV1']
Combined,Combined,2022-04-04,['118522'],['02671' '02670' '02283' '02238' '02672' '02231' '02230' '02673' '02235' '02236' '02237' '02232' '02888' '02857' '02250' '02251' '02957' '02234' '02233' '02956' '02332' '02333' '02334' '02340' '02677' '02676' '02336' '02337' '02339' '02338' '02669' '02678' '02208' '02171' '02202' '02335' '02654' '02653' '02675' '02674' '02900' '02899' '02898' '02897' '02903' '02904' '02905' '02906'],['114737' '' '114728'],['113600' '111478'],['116021'],['116045'],['116044'],['COV1']
Combined,Combined,2022-04-05,['118522'],['02866' '02867' '02901' '02902' '02648' '02649' '02650' '02651' '02644' '02645' '02646' '02647' '02617' '02616' '02642' '02643' '03395' '03373' '03374' '03375' '02435' '02434' '03234' '03237' '03238' '03240' '02436' '02437' '03084' '03083' '03236' '02441' '03235' '02440' '03239' '03241' '02433' '03242' '03243' '02442' '03379' '03372' '03403' '03380' '02367' '02366' '03385' '03397' '03378' '03377' '02439' '02438' '01734' '01733' '02016' '02015' '02031' '02030' '01732' '01731' '02002' '02003' '02004' '02005' '02037' '02036' '02000' '02001'],['114728' '114721'],['113600'],['116021'],['116045'],['116044'],['COV1']
Combined,Combined,2022-04-06,['118522'],['02021' '02022' '02023' '02024' '02017' '02018' '02019' '02020' '01996' '01740' '01739' '01997' '01998' '01999' '01804' '01735' '01736' '01907' '01805' '01903' '01906' '01902' '01737' '01905' '01904' '01738' '01901' '01900' '01880' '01879' '01681' '03253' '01909' '01908' '02420' '02416' '02206' '02710' '02415' '02417' '02422' '02421' '02067' '02935' '02414' '02413' '03284' '01963' '02300' '02483' '02419' '02418'],['114721' '114737'],['113600'],['116021'],['116045'],['116044'],['COV1']


XPCR Module Serial,Run Number,Date,Cartridge Lot,Cartridge Serial,Test Strip Lot,Extraction Plate Lot,Buffer Lot,Release Lot,Wash Lot,Result Code
,,,,,,,,,,
,,,,,,,,,,
V1035,Combined,Combined,['118522'],['03389' '03390' '03394' '03187' '03189' '03193' '02230' '02232' '02233' '02340' '02338' '02335' '02899' '02904' '02867' '02649' '02645' '02616' '03373' '03237' '03083' '03241' '03372' '03397' '02015' '02003' '02036' '02022' '02018' '01903' '01901' '01909' '02415' '02414' '02419'],['114737' '114728' '114721'],['113600'],['116021'],['116045'],['116044'],['COV1']
V1086,Combined,Combined,['118522'],['03387' '03393' '03386' '03184' '03151' '03190' '02283' '02235' '02250' '02332' '02336' '02208' '02898' '02905' '02901' '02650' '02646' '02642' '03374' '03238' '03236' '03242' '03403' '03378' '02031' '02004' '02000' '02023' '02019' '01997' '01906' '01904' '01879' '02416' '02421' '02300'],['114737' '' '114728' '114721'],['113600'],['116021'],['116045'],['116044'],['COV1']
V1088,Combined,Combined,['118522'],['03396' '03391' '03376' '03186' '03188' '03192' '02231' '02237' '02234' '02334' '02339' '02202' '02900' '02903' '02866' '02648' '02644' '02617' '03395' '03234' '03084' '03239' '03379' '03385' '02016' '02002' '02037' '02021' '02017' '01996' '01999' '01902' '01900' '01908' '02417' '02413' '02418'],['114737' '114728' '114721'],['113600'],['116021'],['116045'],['116044'],['COV1']
V1090,Combined,Combined,['118522'],['03388' '03392' '03384' '03185' '03152' '03191' '02238' '02236' '02251' '02333' '02337' '02171' '02897' '02906' '02902' '02651' '02647' '02643' '03375' '03240' '03235' '03243' '03380' '03377' '02030' '02005' '02001' '02024' '02020' '01998' '01907' '01905' '01880' '02420' '02422' '01963'],['114737' '114728' '114721'],['113600'],['116021'],['116045'],['116044'],['COV1']
V1143,Combined,Combined,['118522'],['02893' '02896' '02890' '02892' '02671' '02672' '02888' '02957' '02677' '02669' '02654' '02675' '02435' '02436' '02441' '02433' '02367' '02439' '01734' '01732' '01740' '01804' '01736' '01737' '01681' '02206' '02067' '03284'],['114737' '114728' '114721'],['113600' '111478'],['116021'],['116045'],['116044'],['COV1']
V981,Combined,Combined,['118522'],['02894' '02895' '02889' '02891' '02670' '02673' '02857' '02956' '02676' '02678' '02653' '02674' '02434' '02437' '02440' '02442' '02366' '02438' '01733' '01731' '01739' '01735' '01805' '01738' '03253' '02710' '02935' '02483'],['114737' '114728' '114721'],['113600' '111478'],['116021'],['116045'],['116044'],['COV1']


XPCR Module Serial,Run Number,Date,Cartridge Lot,Cartridge Serial,Test Strip Lot,Extraction Plate Lot,Buffer Lot,Release Lot,Wash Lot,Result Code
,,,,,,,,,,
,,,,,,,,,,
V1035,Combined,Combined,['118522'],['03389' '03390' '03394' '03187' '03189' '03193' '02230' '02232' '02233' '02340' '02338' '02335' '02899' '02904' '02867' '02649' '02645' '02616' '03373' '03237' '03083' '03241' '03372' '03397' '02015' '02003' '02036' '02022' '02018' '01903' '01901' '01909' '02415' '02414' '02419'],['114737' '114728' '114721'],['113600'],['116021'],['116045'],['116044'],['COV1']
V1086,Combined,Combined,['118522'],['03387' '03393' '03386' '03184' '03151' '03190' '02283' '02235' '02250' '02332' '02336' '02208' '02898' '02905' '02901' '02650' '02646' '02642' '03374' '03238' '03236' '03242' '03403' '03378' '02031' '02004' '02000' '02023' '02019' '01997' '01906' '01904' '01879' '02416' '02421' '02300'],['114737' '' '114728' '114721'],['113600'],['116021'],['116045'],['116044'],['COV1']
V1088,Combined,Combined,['118522'],['03396' '03391' '03376' '03186' '03188' '03192' '02231' '02237' '02234' '02334' '02339' '02202' '02900' '02903' '02866' '02648' '02644' '02617' '03395' '03234' '03084' '03239' '03379' '03385' '02016' '02002' '02037' '02021' '02017' '01996' '01999' '01902' '01900' '01908' '02417' '02413' '02418'],['114737' '114728' '114721'],['113600'],['116021'],['116045'],['116044'],['COV1']
V1090,Combined,Combined,['118522'],['03388' '03392' '03384' '03185' '03152' '03191' '02238' '02236' '02251' '02333' '02337' '02171' '02897' '02906' '02902' '02651' '02647' '02643' '03375' '03240' '03235' '03243' '03380' '03377' '02030' '02005' '02001' '02024' '02020' '01998' '01907' '01905' '01880' '02420' '02422' '01963'],['114737' '114728' '114721'],['113600'],['116021'],['116045'],['116044'],['COV1']
V1143,Combined,Combined,['118522'],['02893' '02896' '02890' '02892' '02671' '02672' '02888' '02957' '02677' '02669' '02654' '02675' '02435' '02436' '02441' '02433' '02367' '02439' '01734' '01732' '01740' '01804' '01736' '01737' '01681' '02206' '02067' '03284'],['114737' '114728' '114721'],['113600' '111478'],['116021'],['116045'],['116044'],['COV1']
V981,Combined,Combined,['118522'],['02894' '02895' '02889' '02891' '02670' '02673' '02857' '02956' '02676' '02678' '02653' '02674' '02434' '02437' '02440' '02442' '02366' '02438' '01733' '01731' '01739' '01735' '01805' '01738' '03253' '02710' '02935' '02483'],['114737' '114728' '114721'],['113600' '111478'],['116021'],['116045'],['116044'],['COV1']


XPCR Module Serial,Run Number,Date,Cartridge Lot,Cartridge Serial,Test Strip Lot,Extraction Plate Lot,Buffer Lot,Release Lot,Wash Lot,Result Code
,,,,,,,,,,
,,,,,,,,,,
V1035,1,2022-04-01,['118522'],['03389'],['114737'],['113600'],['116021'],['116045'],['116044'],['COV1']
V1035,2,2022-04-01,['118522'],['03390'],['114737'],['113600'],['116021'],['116045'],['116044'],['COV1']
V1035,3,2022-04-01,['118522'],['03394'],['114737'],['113600'],['116021'],['116045'],['116044'],['COV1']
V1035,4,2022-04-01,['118522'],['03187'],['114737'],['113600'],['116021'],['116045'],['116044'],['COV1']
V1035,5,2022-04-01,['118522'],['03189'],['114737'],['113600'],['116021'],['116045'],['116044'],['COV1']
V1035,6,2022-04-01,['118522'],['03193'],['114737'],['113600'],['116021'],['116045'],['116044'],['COV1']
V1035,7,2022-04-04,['118522'],['02230'],['114737'],['113600'],['116021'],['116045'],['116044'],['COV1']
V1035,8,2022-04-04,['118522'],['02232'],['114737'],['113600'],['116021'],['116045'],['116044'],['COV1']


XPCR Module Serial,Run Number,Date,Cartridge Lot,Cartridge Serial,Test Strip Lot,Extraction Plate Lot,Buffer Lot,Release Lot,Wash Lot,Result Code
,,,,,,,,,,
,,,,,,,,,,
V1035,1,2022-04-01,['118522'],['03389'],['114737'],['113600'],['116021'],['116045'],['116044'],['COV1']
V1035,2,2022-04-01,['118522'],['03390'],['114737'],['113600'],['116021'],['116045'],['116044'],['COV1']
V1035,3,2022-04-01,['118522'],['03394'],['114737'],['113600'],['116021'],['116045'],['116044'],['COV1']
V1035,4,2022-04-01,['118522'],['03187'],['114737'],['113600'],['116021'],['116045'],['116044'],['COV1']
V1035,5,2022-04-01,['118522'],['03189'],['114737'],['113600'],['116021'],['116045'],['116044'],['COV1']
V1035,6,2022-04-01,['118522'],['03193'],['114737'],['113600'],['116021'],['116045'],['116044'],['COV1']
V1035,7,2022-04-04,['118522'],['02230'],['114737'],['113600'],['116021'],['116045'],['116044'],['COV1']
V1035,8,2022-04-04,['118522'],['02232'],['114737'],['113600'],['116021'],['116045'],['116044'],['COV1']


In [7]:
##Define Color Dictionary to use for PCR curves & other plots for up to 36 categories
ColorDict36  = {1:'#FF0000',#Red 1
              2:'#00B050',#Green 2
              3:'#0070C0',#Blue 3
              4:'#7030A0',#Purple 4
              5:'#808080',#Light Grey 5
              6:'#FF6600',#Orange 6
              7:'#FFCC00',#Yellow 7
              8:'#9999FF',#Light Purple 8
              9:'#333333',#Black 9
              10:'#808000',#Goldish 10
              11:'#FF99CC',#Hot Pink 11
              12:'#003300',#Dark Green 12
              13:'#FF0000',#Red 1
              14:'#00B050',#Green 2
              15:'#0070C0',#Blue 3
              16:'#7030A0',#Purple 4
              17:'#808080',#Light Grey 5
              18:'#FF6600',#Orange 6
              19:'#FFCC00',#Yellow 7
              20:'#9999FF',#Light Purple 8
              21:'#333333',#Black 9
              22:'#808000',#Goldish 10
              23:'#FF99CC',#Hot Pink 11
              24:'#003300',#Dark Green 12  
              25:'#FF0000',#Red 1
              26:'#00B050',#Green 2
              27:'#0070C0',#Blue 3
              28:'#7030A0',#Purple 4
              29:'#808080',#Light Grey 5
              30:'#FF6600',#Orange 6
              31:'#FFCC00',#Yellow 7
              32:'#9999FF',#Light Purple 8
              33:'#333333',#Black 9
              34:'#808000',#Goldish 10
              35:'#FF99CC',#Hot Pink 11
              36:'#003300',#Dark Green 12  
            }

styleDict36 = {1: 'solid',
              2: 'solid',
              3: 'solid',
              4: 'solid',
              5: 'solid',
              6: 'solid',
              7: 'solid',
              8: 'solid',
              9: 'solid',
              10: 'solid',
              11: 'solid',
              12: 'solid',
              13: 'dashed',
              14: 'dashed',
              15: 'dashed',
              16: 'dashed',
              17: 'dashed',
              18: 'dashed',
              19: 'dashed',
              20: 'dashed',
              21: 'dashed',
              22: 'dashed',
              23: 'dashed',
              24: 'dashed',
              25: 'dashdot',
              26: 'dashdot',
              27: 'dashdot',
              28: 'dashdot',
              29: 'dashdot',
              30: 'dashdot',
              31: 'dashdot',
              32: 'dashdot',
              33: 'dashdot',
              34: 'dashdot',
              35: 'dashdot',
              36: 'dashdot'}

In [ ]:
##Step 2: Plot Amplification Curves for All Samples Included in Testing

##Define Modules to Include in Plots
mods = sorted(COC_PCR.index.unique(0).dropna().to_list(), key= lambda x: int(x.replace("V", "").replace("EXP", "")))#Define Channels to Include in Plots

##Exclude samples by Test Guid
exclude_list = []

##Define Lots to Plot
COC_PCR['Cartridge Lot'] = COC_PCR['Pcr Cartridge Barcode'].str[18:24]
lots = COC_PCR['Cartridge Lot'].unique().tolist()

#Define Channels to Plot
channels = ['Yellow', 'Green', 'Far_Red']

#Define Targets to Include in Plots
target_names = ['N gene', 'Nsp2 Gene', 'SPC2']

#Define Target / Channel Combinations
targets = {'Green':'Nsp2 Gene',
           'Yellow':'N gene',
           'Far_Red':'SPC2'}

channelDataDict = {}
channelData = {}    

channelData['Green'] = generateSummaryTable(NormDataDict_PCR['Green'], 'Green')
channelData['Yellow'] =  generateSummaryTable(NormDataDict_PCR['Yellow'], 'Yellow')
channelData['Far_Red'] =  generateSummaryTable(NormDataDict_PCR['Far_Red'], 'Far_Red')



fig, axs = plt.subplots(len(channels), len(lots)+1, figsize=(10,10),sharey='row',sharex=True)
plt.subplots_adjust(wspace=0.05,hspace=0.35)

for channel in channels:

    channelData[channel][1].loc[:,'Cartridge Lot'] = channelData[channel][1].loc[:,'Cartridge Barcode'].str[18:24]

    channelplotdata = channelData[channel][1]#.join(treatment_key)
    channelplotdata = channelplotdata.loc[~channelplotdata['Test Guid'].isin(exclude_list),:]
    channelplotdata.reset_index(inplace=True)
    channelplotdata.set_index(['XPCR Module Serial', 'Run Number', 'Cartridge Lane'],inplace=True)
    
    channelDataDict[channel] = channelplotdata.set_index(['Cartridge Lot'],append=True).loc[:,['Ct', 'Blank Reading', 'Dark Reading', 'Readings 5', 'End Point Fluorescence', 'Max Peak Height', 'EPR', 'Target Result', 'Test Guid', 'Replicate Result']]
    channelDataDict[channel].columns = pd.MultiIndex.from_product([[channel],channelDataDict[channel].columns])
    
    ##Remove Entries that do not have a XPCR Module (i.e. No Result Samples)
    if np.nan in channelplotdata.index.unique(0):
        channelplotdata.drop(np.nan,inplace=True,axis=0)
    
    channelplotdata.sort_values('Readings 45', ascending=True, inplace=True)
    
    #Plot each line in channelplotdata 
    for mod, run, lane in channelplotdata.index.unique():
                
        if mod in mods:
            guid = channelplotdata.loc[(mod,run,lane), 'Test Guid']
            
            lot = channelplotdata.loc[(mod,run,lane), 'Cartridge Lot']
            target = channelplotdata.loc[(mod,run,lane), 'Target Name']
            
            #Ensure that iteration guid is not in exclude list, if it is not then plot the data.
            if guid not in exclude_list:
                X = np.arange(1,46,1)
                Y = channelplotdata.loc[(mod,run,lane), ['Readings '+str(read) for read in range(1,46,1)]]
                sns.lineplot(x=X,y=Y, color=ColorDict36[mods.index(mod)+1],linestyle=styleDict36[mods.index(mod)+1],ax=axs[channels.index(channel),lots.index(lot)])
                axs[0,lots.index(lot)].set_title(lot,fontsize=10, color='k')
                
       
    for lot in lots:
        if channels.index(channel) == len(channels)-1:
                axs[channels.index(channel),lots.index(lot)].set_xlabel('cycle')
            
    axs[channels.index(channel),0].set_ylabel(targets[channel])

legend_elements = []
for mod in mods:
    newLabel = Line2D([0], [0], color=ColorDict36[mods.index(mod)+1], linestyle=styleDict36[mods.index(mod)+1], lw=3,label=mod)
    legend_elements.append(newLabel)

axs[0,len(lots)].legend(handles=legend_elements, fontsize=18,loc='upper left',ncol=1)


## Use this if curves end up upside down, happens from time to time depending on how negative results look.
# for row in range(0,len(channels)):
#     axs[row,0].invert_yaxis()
    


for row in range(0,len(channels)):
    axs[row,len(lots)].axis('Off')
plt.show()
print("_ _"*30)
data = pd.concat([channelDataDict[df] for df in channelDataDict],axis=1).sort_index()

In [ ]:
##Step 3A Get Overall Results Summary By Cartridge Lot
from statsmodels.stats.proportion import proportion_confint
COC_PCR['UNR'] = np.where(COC_PCR['Overall Result']=='Unresolved', 1, 0)
COC_PCR['POS'] = np.where(COC_PCR['Overall Result']=='Positive', 1, 0)
COC_PCR['NEG'] = np.where(COC_PCR['Overall Result']=='Negative', 1, 0)
COC_PCR['IND'] = np.where(COC_PCR['Overall Result']=='Indeterminate', 1, 0)
COC_PCR['NR'] = np.where(COC_PCR['Overall Result']=='NoResult', 1, 0)

Overall_Results_Summary = COC_PCR[['UNR',
                'POS',
                'NEG',
                'IND',
                'NR',
                'Cartridge Lot']].groupby(['Cartridge Lot']).agg({'NEG':['count','sum','mean'],
                                                                                        'POS':['sum','mean'],
                                                                                        'UNR':['sum','mean'],
                                                                                        'IND':['sum','mean'],
                                                                                        'NR':['sum','mean']})

Overall_Results_Summary.columns = ['Sample Size',
                                   'NEG #',
                                   'NEG %',
                                   'POS #',
                                   'POS %',
                                   'UNR #',
                                   'UNR %',
                                   'IND #',
                                   'IND %',
                                   'NR #',
                                   'NR %']

display(Overall_Results_Summary.style\
                       .format(subset=[x for x in Overall_Results_Summary.columns if '%' in x], formatter='{:.2%}')\
                       .format(subset=[x for x in Overall_Results_Summary.columns if '#' in x], formatter='{:.0f}'))


for item in Overall_Results_Summary.index:

    lcb, ucb = proportion_confint(count=Overall_Results_Summary.loc[item,'UNR #'], nobs=Overall_Results_Summary.loc[item,'Sample Size'], alpha=(1-0.95), method='beta')
    Overall_Results_Summary.loc[item, ['UNR 95% CI LCB', 'UNR 95% CI UCB']] = [lcb, ucb]
    
    lcb, ucb = proportion_confint(count=Overall_Results_Summary.loc[item,'NEG #'], nobs=Overall_Results_Summary.loc[item,'Sample Size'], alpha=(1-0.95), method='beta')
    Overall_Results_Summary.loc[item, ['NEG 95% CI LCB', 'NEG 95% CI UCB']] = [lcb, ucb]

display(Overall_Results_Summary.loc[:, [x for x in Overall_Results_Summary.columns if 'UNR' in x or 'Size' in x]].sort_index(axis=1).style\
                       .format(subset=[x for x in Overall_Results_Summary.columns if '%' in x and ('UNR' in x or 'Size' in x)], formatter='{:.2%}')\
                       .format(subset=[x for x in Overall_Results_Summary.columns if '#' in x and ('UNR' in x or 'Size' in x)], formatter='{:.0f}'))
##Step 3B Get Overall Results Summary By Cartridge Lot / N500 Serial Number

Overall_Results_Summary = COC_PCR[['UNR',
                'POS',
                'NEG',
                'IND',
                'NR',
                'Cartridge Lot', 'N500 Serial Number']].groupby(['Cartridge Lot', 'N500 Serial Number']).agg({'NEG':['count','sum','mean'],
                                                                                        'POS':['sum','mean'],
                                                                                        'UNR':['sum','mean'],
                                                                                        'IND':['sum','mean'],
                                                                                        'NR':['sum','mean']})

Overall_Results_Summary.columns = ['Sample Size',
                                   'NEG #',
                                   'NEG %',
                                   'POS #',
                                   'POS %',
                                   'UNR #',
                                   'UNR %',
                                   'IND #',
                                   'IND %',
                                   'NR #',
                                   'NR %']


display(Overall_Results_Summary.style\
                       .format(subset=[x for x in Overall_Results_Summary.columns if '%' in x], formatter='{:.2%}')\
                       .format(subset=[x for x in Overall_Results_Summary.columns if '#' in x], formatter='{:.0f}'))

for item in Overall_Results_Summary.index:

    lcb, ucb = proportion_confint(count=Overall_Results_Summary.loc[item,'UNR #'], nobs=Overall_Results_Summary.loc[item,'Sample Size'], alpha=(1-0.95), method='beta')
    Overall_Results_Summary.loc[item, ['UNR 95% CI LCB', 'UNR 95% CI UCB']] = [lcb, ucb]
    
    lcb, ucb = proportion_confint(count=Overall_Results_Summary.loc[item,'NEG #'], nobs=Overall_Results_Summary.loc[item,'Sample Size'], alpha=(1-0.95), method='beta')
    Overall_Results_Summary.loc[item, ['NEG 95% CI LCB', 'NEG 95% CI UCB']] = [lcb, ucb]

display(Overall_Results_Summary.loc[:, [x for x in Overall_Results_Summary.columns if 'UNR' in x or 'Size' in x]].sort_index(axis=1).style\
                       .format(subset=[x for x in Overall_Results_Summary.columns if '%' in x and ('UNR' in x or 'Size' in x)], formatter='{:.2%}')\
                       .format(subset=[x for x in Overall_Results_Summary.columns if '#' in x and ('UNR' in x or 'Size' in x)], formatter='{:.0f}'))
##Step 3C Get Overall Results Summary By Cartridge Lot / PCR Module Serial Number

Overall_Results_Summary = COC_PCR.reset_index()[['UNR',
                'POS',
                'NEG',
                'IND',
                'NR',
                'Cartridge Lot', 'PCR Module Serial']].groupby(['Cartridge Lot', 'PCR Module Serial']).agg({'NEG':['count','sum','mean'],
                                                                                        'POS':['sum','mean'],
                                                                                        'UNR':['sum','mean'],
                                                                                        'IND':['sum','mean'],
                                                                                        'NR':['sum','mean']})

Overall_Results_Summary.columns = ['Sample Size',
                                   'NEG #',
                                   'NEG %',
                                   'POS #',
                                   'POS %',
                                   'UNR #',
                                   'UNR %',
                                   'IND #',
                                   'IND %',
                                   'NR #',
                                   'NR %']





display(Overall_Results_Summary.style\
                       .format(subset=[x for x in Overall_Results_Summary.columns if '%' in x], formatter='{:.2%}')\
                       .format(subset=[x for x in Overall_Results_Summary.columns if '#' in x], formatter='{:.0f}'))

for item in Overall_Results_Summary.index:

    lcb, ucb = proportion_confint(count=Overall_Results_Summary.loc[item,'UNR #'], nobs=Overall_Results_Summary.loc[item,'Sample Size'], alpha=(1-0.95), method='beta')
    Overall_Results_Summary.loc[item, ['UNR 95% CI LCB', 'UNR 95% CI UCB']] = [lcb, ucb]
    
    lcb, ucb = proportion_confint(count=Overall_Results_Summary.loc[item,'NEG #'], nobs=Overall_Results_Summary.loc[item,'Sample Size'], alpha=(1-0.95), method='beta')
    Overall_Results_Summary.loc[item, ['NEG 95% CI LCB', 'NEG 95% CI UCB']] = [lcb, ucb]


display(Overall_Results_Summary.loc[:, [x for x in Overall_Results_Summary.columns if 'UNR' in x or 'Size' in x]].sort_index(axis=1).style\
                       .format(subset=[x for x in Overall_Results_Summary.columns if '%' in x and ('UNR' in x or 'Size' in x)], formatter='{:.2%}')\
                       .format(subset=[x for x in Overall_Results_Summary.columns if '#' in x and ('UNR' in x or 'Size' in x)], formatter='{:.0f}'))

In [ ]:
##Step 4 Get Line Data
COC_PCR['Condition'] = 'Negative UVT'
COC_PCR['Cartridge Lot'] = COC_PCR['Pcr Cartridge Barcode'].str[18:24]
COC_PCR['Cartridge Serial'] = COC_PCR['Pcr Cartridge Barcode'].str[28:32]
COC_PCR['Extraction Plate Lot'] = COC_PCR['Capture Plate Barcode'].str[18:24]
COC_PCR['Test Strip Lot'] = COC_PCR['Test Strip NeuMoDx Barcode'].str[18:24]
COC_PCR['Buffer Lot'] = COC_PCR['Buffer Barcode'].str[18:24]
COC_PCR['Wash Lot'] = COC_PCR['Wash Reagent Barcode'].str[18:24]
COC_PCR['Release Lot'] = COC_PCR['Release Reagent Barcode'].str[18:24]

##Get Target Results from data
TargetResults = data.reset_index().set_index(('Yellow', 'Test Guid')).loc[:,(slice(None),'Target Result')]
TargetResults.columns = ['Yellow Target Result', 'Green Target Result', 'Far Red Target Result']
TargetResults.replace({'TargetAmplified':'AMP',
                       'TargetNotAmplified':'NotAmp',
                       'TargetIndeterminate':'IND',
                       'TargetUnresolved':'UNR',
                       'NoResult':'NR'},inplace=True)
TargetResults.index.names = ['Test Guid']

EPRs = data.reset_index().set_index(('Yellow', 'Test Guid')).loc[:,(slice(None),'EPR')]
EPRs.columns = ['Yellow Target EPR', 'Green Target EPR', 'Far Red Target EPR']
EPRs.index.names = ['Test Guid']
MPHs = data.reset_index().set_index(('Yellow', 'Test Guid')).loc[:,(slice(None),'Max Peak Height')]
MPHs.columns = ['Yellow Target Max Peak Height', 'Green Target Max Peak Height',  'Far Red Target Max Peak Height']
MPHs.index.names = ['Test Guid']

LineData = COC_PCR.reset_index().set_index('Test Guid').join(TargetResults).join(EPRs).join(MPHs).loc[:,['N500 Serial Number', 'PCR Module Serial', 'Pcr Cartridge Lane', 'Condition', 'Sample ID',
                       'Start Date Time', 'Cartridge Lot', 'Cartridge Serial', 'Overall Result',
                       'Yellow Target Result', 'Yellow Target Ct', 'Yellow Target EP', 'Yellow Target EPR', 'Yellow Target Max Peak Height',
                       'Green Target Result', 'Green Target Ct', 'Green Target EP', 'Green Target EPR', 'Green Target Max Peak Height',
                       'Far Red Target Result','Far Red Target Ct', 'Far Red Target EP', 'Far Red Target EPR', 'Far Red Target Max Peak Height',
                       'Buffer Lot', 'Extraction Plate Lot', 'Test Strip Lot', 'Wash Lot', 'Release Lot']]\
                        .sort_values(['Cartridge Serial','Pcr Cartridge Lane'])


LineData.style.format({'Start Date Time':'{:%Y-%m-%d %H:%M:%S}',
                       'Pcr Cartridge Lane':'{:.0f}',
                       'Yellow Target Ct':'{:.2f}',
                       'Yellow Target EP':'{:.0f}',
                       'Yellow Target Max Peak Height':'{:.0f}',
                       'Yellow Target EPR':'{:.2f}',
                       'Green Target Ct':'{:.2f}',
                       'Green Target EP':'{:.0f}',
                       'Green Target Max Peak Height':'{:.0f}',
                       'Green Target EPR':'{:.2f}',
                       'Orange Target Ct':'{:.2f}',
                       'Orange Target EP':'{:.0f}',
                       'Green Target Max Peak Height':'{:.0f}',
                       'Green Target EPR':'{:.2f}',
                       'Far Red Target Ct':'{:.2f}',
                       'Far Red Target EP':'{:.0f}',
                       'Far Red Target Max Peak Height':'{:.0f}',
                       'Far Red Target EPR':'{:.2f}',
                       'Red Target Ct':'{:.2f}',
                       'Red Target EP':'{:.0f}',
                       'Red Target Max Peak Height':'{:.0f}',
                       'Red Target EPR':'{:.2f}',
                       
                       },na_rep='-').hide_index()

In [ ]:
##Step 5 Get Line Data (with Flags) for invalid Samples

TargetResults.index.names = ['Test Guid']
TargetResults


LineDataInvalids = COC_PCR.reset_index().set_index('Test Guid').join(TargetResults).loc[:,['N500 Serial Number', 'PCR Module Serial', 'Pcr Cartridge Lane', 'Condition', 'Sample ID',
                       'Start Date Time', 'Cartridge Lot', 'Cartridge Serial', 'Overall Result',
                       'Yellow Target Result', 'Yellow Target Ct', 'Yellow Target EP', 'Yellow Target Flag',
                       'Green Target Result', 'Green Target Ct', 'Green Target EP', 'Green Target Flag',
                       'Far Red Target Result','Far Red Target Ct', 'Far Red Target EP', 'Far Red Target Flag',
                       'Buffer Lot', 'Extraction Plate Lot', 'Test Strip Lot', 'Wash Lot', 'Release Lot']]\
                        .sort_values(['Cartridge Serial','Pcr Cartridge Lane'])

LineDataInvalids = LineDataInvalids.loc[~LineDataInvalids['Overall Result'].isin(['Positive', 'Negative'])]
LineDataInvalids.style.format({'Start Date Time':'{:%Y-%m-%d %H:%M:%S}',
                       'Pcr Cartridge Lane':'{:.0f}',
                       'Yellow Target Ct':'{:.2f}',
                       'Yellow Target EP':'{:.0f}',
                       'Green Target Ct':'{:.2f}',
                       'Green Target EP':'{:.0f}',
                       'Orange Target Ct':'{:.2f}',
                       'Orange Target EP':'{:.0f}',
                       'Far Red Target Ct':'{:.2f}',
                       'Far Red Target EP':'{:.0f}',
                       'Red Target Ct':'{:.2f}',
                       'Red Target EP':'{:.0f}',
                       
                       },na_rep='-').hide_index()

In [ ]:
#Step 6 Plot Amplification Curves for Invalid Samples
invalid_samples = LineDataInvalids.index.unique(0).to_list()
##Define Modules to Include in Plots
mods = sorted(COC_PCR.index.unique(0).dropna().to_list(), key= lambda x: int(x.replace("V", "").replace("EXP", "")))#Define Channels to Include in Plots

##Exclude samples by Test Guid
exclude_list = []

##Define Lots to Plot
COC_PCR['Cartridge Lot'] = COC_PCR['Pcr Cartridge Barcode'].str[18:24]
lots = COC_PCR['Cartridge Lot'].unique().tolist()

#Define Channels to Plot
channels = ['Yellow', 'Green', 'Far_Red']

#Define Targets to Include in Plots
target_names = ['N gene', 'Nsp2 Gene', 'SPC2']

#Define Target / Channel Combinations
targets = {'Green':'Nsp2 Gene',
           'Yellow':'N gene',
           'Far_Red':'SPC2'}

channelDataDict = {}
channelData = {}    

channelData['Green'] = generateSummaryTable(NormDataDict_PCR['Green'], 'Green')
channelData['Yellow'] =  generateSummaryTable(NormDataDict_PCR['Yellow'], 'Yellow')
channelData['Far_Red'] =  generateSummaryTable(NormDataDict_PCR['Far_Red'], 'Far_Red')
consumables = generateLotTable(COC_PCR)
plt.subplots_adjust(wspace=0.05)

fig, axs = plt.subplots(len(channels), len(lots)+1, figsize=(10,10),sharey='row',sharex=True)
plt.subplots_adjust(wspace=0.05,hspace=0.35)

for channel in channels:

    channelData[channel][1].loc[:,'Cartridge Lot'] = channelData[channel][1].loc[:,'Cartridge Barcode'].str[18:24]

    channelplotdata = channelData[channel][1]
    channelplotdata = channelplotdata.loc[channelplotdata['Test Guid'].isin(LineDataInvalids.index),:]
    channelplotdata.reset_index(inplace=True)
    channelplotdata.set_index(['XPCR Module Serial', 'Run Number', 'Cartridge Lane'],inplace=True)
    
    channelDataDict[channel] = channelplotdata.set_index(['Cartridge Lot'],append=True).loc[:,['Ct', 'Blank Reading', 'Dark Reading', 'Readings 5', 'End Point Fluorescence', 'Max Peak Height', 'EPR', 'Target Result', 'Test Guid', 'Replicate Result']]
    channelDataDict[channel].columns = pd.MultiIndex.from_product([[channel],channelDataDict[channel].columns])
    
    ##Remove Entries that do not have a XPCR Module (i.e. No Result Samples)
    if np.nan in channelplotdata.index:
        channelplotdata.drop(np.nan,inplace=True,axis=0)
    
    
    #Plot each line in channelplotdata 
    for mod, run, lane in channelplotdata.index.unique():
                
        if mod in mods:
            guid = channelplotdata.loc[(mod,run,lane), 'Test Guid']
            
            lot = channelplotdata.loc[(mod,run,lane), 'Cartridge Lot']
            target = channelplotdata.loc[(mod,run,lane), 'Target Name']
            
            #Ensure that iteration guid is not in exclude list, if it is not then plot the data.
            if guid not in exclude_list:
                X = np.arange(1,46,1)
                Y = channelplotdata.loc[(mod,run,lane), ['Readings '+str(read) for read in range(1,46,1)]]
                sns.lineplot(x=X,y=Y, color=ColorDict36[invalid_samples.index(guid)+1],linestyle=styleDict36[mods.index(mod)+1],ax=axs[channels.index(channel),lots.index(lot)])
                axs[0,lots.index(lot)].set_title(lot,fontsize=10, color='k')
                
       
    for lot in lots:
        if channels.index(channel) == len(channels)-1:
                axs[channels.index(channel),lots.index(lot)].set_xlabel('cycle')
            
    axs[channels.index(channel),0].set_ylabel(targets[channel])

legend_elements = []
for sample in invalid_samples:
    sampleInfo = COC_PCR[COC_PCR['Test Guid']==sample]
    sampleString = (sampleInfo.index.unique(0).values[0] + " Lane " +
                    str(int(sampleInfo.index.unique(1).values[0]))+"\nCartridge Serial: "+
                    sampleInfo['Cartridge Serial'].values[0]+"\nSample ID: "+
                    
                    sampleInfo['Sample ID'].values[0]
                   )
    newLabel = Line2D([0], [0], color=ColorDict36[invalid_samples.index(sample)+1], linestyle=styleDict36[invalid_samples.index(sample)+1], lw=3,label=sampleString)
    legend_elements.append(newLabel)

axs[0,len(lots)].legend(handles=legend_elements, fontsize=12,loc='upper left',ncol=2)


## Use this if curves end up upside down, happens from time to time depending on how negative results look.
# for row in range(0,len(channels)):
#     axs[row,0].invert_yaxis()
    


for row in range(0,len(channels)):
    axs[row,len(lots)].axis('Off')
plt.show()


In [ ]:
#Step 7A: Get Samples that are either positive calls, or have weak amplification
## Use this to Get Guids of samples that have weak amplification calls but no EP / Ct calls and are called 
NormDataDict_PCR['Green'].reset_index()[['XPCR Module Serial','Test Guid','Readings 45', 'End Point Fluorescence','Target Result']].sort_values('Readings 45',ascending=False)
NormDataDict_PCR['Yellow'].reset_index()[['XPCR Module Serial','Test Guid','Readings 45', 'End Point Fluorescence','Target Result']].sort_values('Readings 45',ascending=False)

In [ ]:
##Step 7B: Copy and paste test guids into weakamp_or_positive_list
##to generate linedata for samples that are either false positive or show signs of weak amplification
PositiveWeakAmpSamples = LineData.loc[['0170122e-08b5-ec11-85c7-5cf3709f02dd','889fe60f-6ab4-ec11-85c7-5cf3709f02dd','f47d36e7-35b4-ec11-85c7-5cf3709f02dd']]
PositiveWeakAmpSamples.style.format({'Start Date Time':'{:%Y-%m-%d %H:%M:%S}',
                       'Pcr Cartridge Lane':'{:.0f}',
                       'Yellow Target Ct':'{:.2f}',
                       'Yellow Target EP':'{:.0f}',
                       'Green Target Ct':'{:.2f}',
                       'Green Target EP':'{:.0f}',
                       'Orange Target Ct':'{:.2f}',
                       'Orange Target EP':'{:.0f}',
                       'Far Red Target Ct':'{:.2f}',
                       'Far Red Target EP':'{:.0f}',
                       'Red Target Ct':'{:.2f}',
                       'Red Target EP':'{:.0f}',
                       
                       },na_rep='-').hide_index()

In [ ]:
##Step 7C: Plot Data from samples selected from Step 7B
#Step 7 Plot Amplification Curves for Positive / Weak Amp Samples
invalid_samples = PositiveWeakAmpSamples.index.unique(0).to_list()
##Define Modules to Include in Plots
mods = sorted(COC_PCR.index.unique(0).dropna().to_list(), key= lambda x: int(x.replace("V", "").replace("EXP", "")))#Define Channels to Include in Plots

##Exclude samples by Test Guid
exclude_list = []

##Define Lots to Plot
COC_PCR['Cartridge Lot'] = COC_PCR['Pcr Cartridge Barcode'].str[18:24]
lots = COC_PCR['Cartridge Lot'].unique().tolist()

#Define Channels to Plot
channels = ['Yellow', 'Green', 'Far_Red']

#Define Targets to Include in Plots
target_names = ['N gene', 'Nsp2 Gene', 'SPC2']

#Define Target / Channel Combinations
targets = {'Green':'Nsp2 Gene',
           'Yellow':'N gene',
           'Far_Red':'SPC2'}

channelDataDict = {}
channelData = {}    

channelData['Green'] = generateSummaryTable(NormDataDict_PCR['Green'], 'Green')
channelData['Yellow'] =  generateSummaryTable(NormDataDict_PCR['Yellow'], 'Yellow')
channelData['Far_Red'] =  generateSummaryTable(NormDataDict_PCR['Far_Red'], 'Far_Red')
consumables = generateLotTable(COC_PCR)
plt.subplots_adjust(wspace=0.05)

fig, axs = plt.subplots(len(channels), len(lots)+1, figsize=(10,10),sharey='row',sharex=True)
plt.subplots_adjust(wspace=0.05,hspace=0.35)

for channel in channels:

    channelData[channel][1].loc[:,'Cartridge Lot'] = channelData[channel][1].loc[:,'Cartridge Barcode'].str[18:24]

    channelplotdata = channelData[channel][1]
    channelplotdata = channelplotdata.loc[channelplotdata['Test Guid'].isin(invalid_samples),:]
    channelplotdata.reset_index(inplace=True)
    channelplotdata.set_index(['XPCR Module Serial', 'Run Number', 'Cartridge Lane'],inplace=True)
    
    channelDataDict[channel] = channelplotdata.set_index(['Cartridge Lot'],append=True).loc[:,['Ct', 'Blank Reading', 'Dark Reading', 'Readings 5', 'End Point Fluorescence', 'Max Peak Height', 'EPR', 'Target Result', 'Test Guid', 'Replicate Result']]
    channelDataDict[channel].columns = pd.MultiIndex.from_product([[channel],channelDataDict[channel].columns])
    
    ##Remove Entries that do not have a XPCR Module (i.e. No Result Samples)
    if np.nan in channelplotdata.index:
        channelplotdata.drop(np.nan,inplace=True,axis=0)
    
    
    #Plot each line in channelplotdata 
    for mod, run, lane in channelplotdata.index.unique():
                
        if mod in mods:
            guid = channelplotdata.loc[(mod,run,lane), 'Test Guid']
            
            lot = channelplotdata.loc[(mod,run,lane), 'Cartridge Lot']
            target = channelplotdata.loc[(mod,run,lane), 'Target Name']
            
            #Ensure that iteration guid is not in exclude list, if it is not then plot the data.
            if guid not in exclude_list:
                X = np.arange(1,46,1)
                Y = channelplotdata.loc[(mod,run,lane), ['Readings '+str(read) for read in range(1,46,1)]]
                sns.lineplot(x=X,y=Y, color=ColorDict36[invalid_samples.index(guid)+1],linestyle=styleDict36[mods.index(mod)+1],ax=axs[channels.index(channel),lots.index(lot)])
                axs[0,lots.index(lot)].set_title(lot,fontsize=10, color='k')
                
       
    for lot in lots:
        if channels.index(channel) == len(channels)-1:
                axs[channels.index(channel),lots.index(lot)].set_xlabel('cycle')
            
    axs[channels.index(channel),0].set_ylabel(targets[channel])

legend_elements = []
for sample in invalid_samples:
    sampleInfo = COC_PCR[COC_PCR['Test Guid']==sample]
    sampleString = (sampleInfo.index.unique(0).values[0] + " Lane " +
                    str(int(sampleInfo.index.unique(1).values[0]))+"\nCartridge Serial: "+
                    sampleInfo['Cartridge Serial'].values[0]+"\nSample ID: "+
                    
                    sampleInfo['Sample ID'].values[0]
                   )
    newLabel = Line2D([0], [0], color=ColorDict36[invalid_samples.index(sample)+1], linestyle=styleDict36[invalid_samples.index(sample)+1], lw=3,label=sampleString)
    legend_elements.append(newLabel)

axs[0,len(lots)].legend(handles=legend_elements, fontsize=12,loc='upper left',ncol=2)


## Use this if curves end up upside down, happens from time to time depending on how negative results look.
# for row in range(0,len(channels)):
#     axs[row,0].invert_yaxis()
    


for row in range(0,len(channels)):
    axs[row,len(lots)].axis('Off')
plt.show()